# How to make reports from transactional data

I will use https://www.kaggle.com/carrie1/ecommerce-data . If you want to play with it then download the data file and use the next cell to get the data into colab. Or upload it on your google drive and read it from there (google how to do it).

In [0]:
#from google.colab import files
#files.upload()

In [0]:
import pandas as pd

In [0]:
df = pd.read_csv('drive/My Drive/data/ecommerce-data.zip',encoding = "cp1252")

In [0]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


Look at missing values

In [0]:
df.isna().mean()

InvoiceNo      0.000000
StockCode      0.000000
Description    0.002683
Quantity       0.000000
InvoiceDate    0.000000
UnitPrice      0.000000
CustomerID     0.249267
Country        0.000000
dtype: float64

Let's fill these missing customer ids with something to have its sales in the report

In [0]:
df['CustomerID'].fillna(-999,inplace=True)

You can see transactions per user. Let's see what time period we have. But first convert text date to python date

In [0]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'],format='%m/%d/%Y %H:%M')

In [0]:
df['InvoiceDate'].min()

Timestamp('2010-12-01 08:26:00')

In [0]:
df['InvoiceDate'].max()

Timestamp('2011-12-09 12:50:00')

It's about a year. Ok. Now we want to create a sales report by month.

# 1. Create a table with unique users

In the project you already have this table. And in real life there is always such a table in the database. But in this case we do not have it.

In [0]:
user = df.groupby(['CustomerID'])['InvoiceDate'].min().reset_index()

In [0]:
user.head()

,CustomerID,InvoiceDate
0,-999.0,2010-12-01 11:52:00
1,12346.0,2011-01-18 10:01:00
2,12347.0,2010-12-07 14:57:00
3,12348.0,2010-12-16 19:09:00
4,12349.0,2011-11-21 09:51:00


In [0]:
user.columns = ['CustomerID','reg_date']

In [0]:
user['reg_month'] = user['reg_date'].values.astype('datetime64[M]')

No worries). It's just the first link from google on "pandas how to get first day of month" https://stackoverflow.com/questions/45304531/extracting-the-first-day-of-month-of-a-datetime-type-column-in-pandas



In [0]:
user

,CustomerID,reg_date,reg_month
0,-999.0,2010-12-01 11:52:00,2010-12-01
1,12346.0,2011-01-18 10:01:00,2011-01-01
2,12347.0,2010-12-07 14:57:00,2010-12-01
3,12348.0,2010-12-16 19:09:00,2010-12-01
4,12349.0,2011-11-21 09:51:00,2011-11-01
...,...,...,...
4368,18280.0,2011-03-07 09:52:00,2011-03-01
4369,18281.0,2011-06-12 10:53:00,2011-06-01
4370,18282.0,2011-08-05 13:35:00,2011-08-01
4371,18283.0,2011-01-06 14:14:00,2011-01-01


# 2. Generate a table with each possible month for each user.

That is how you get records in your report then users are in churn. Look




In [0]:
min_month = df['InvoiceDate'].values.astype('datetime64[M]').min()

In [0]:
max_month = df['InvoiceDate'].values.astype('datetime64[M]').max()

In [0]:
(min_month, max_month)

(numpy.datetime64('2010-12'), numpy.datetime64('2011-12'))

In [0]:
pd.date_range(min_month,max_month,freq='MS')

DatetimeIndex(['2010-12-01', '2011-01-01', '2011-02-01', '2011-03-01',
               '2011-04-01', '2011-05-01', '2011-06-01', '2011-07-01',
               '2011-08-01', '2011-09-01', '2011-10-01', '2011-11-01',
               '2011-12-01'],
              dtype='datetime64[ns]', freq='MS')

MS means start of the month, to get the first day of the month. 'M' gives the last day


In [0]:
dr = pd.DataFrame(pd.date_range(min_month,max_month,freq='MS'))

In [0]:
dr

,0
0,2010-12-01
1,2011-01-01
2,2011-02-01
3,2011-03-01
4,2011-04-01
5,2011-05-01
6,2011-06-01
7,2011-07-01
8,2011-08-01
9,2011-09-01


In [0]:
dr.columns = ['month']

Now we have to perform cross join with user table and get


In [0]:
len(user)*len(dr)

56849

rows. In pandas it could be performed like this

In [0]:
dr['key'] = 1
user['key'] = 1

In [0]:
report = dr.merge(user,on='key')

In [0]:
report.head()

,month,key,CustomerID,reg_date,reg_month
0,2010-12-01,1,-999.0,2010-12-01 11:52:00,2010-12-01
1,2010-12-01,1,12346.0,2011-01-18 10:01:00,2011-01-01
2,2010-12-01,1,12347.0,2010-12-07 14:57:00,2010-12-01
3,2010-12-01,1,12348.0,2010-12-16 19:09:00,2010-12-01
4,2010-12-01,1,12349.0,2011-11-21 09:51:00,2011-11-01


In [0]:
len(report)

56849

Like expected. But we have some extra rows: some users came long after minimal date (2010-12) and it's stupid to have records for a user before he came to us).

In [0]:
len(report[report['month']>=report['reg_month']])

37449

In [0]:
report = report[report['month']>=report['reg_month']]

Let's look at some specific user

In [0]:
report[report['CustomerID'] == 12346.0]

,month,key,CustomerID,reg_date,reg_month
4374,2011-01-01,1,12346.0,2011-01-18 10:01:00,2011-01-01
8747,2011-02-01,1,12346.0,2011-01-18 10:01:00,2011-01-01
13120,2011-03-01,1,12346.0,2011-01-18 10:01:00,2011-01-01
17493,2011-04-01,1,12346.0,2011-01-18 10:01:00,2011-01-01
21866,2011-05-01,1,12346.0,2011-01-18 10:01:00,2011-01-01
26239,2011-06-01,1,12346.0,2011-01-18 10:01:00,2011-01-01
30612,2011-07-01,1,12346.0,2011-01-18 10:01:00,2011-01-01
34985,2011-08-01,1,12346.0,2011-01-18 10:01:00,2011-01-01
39358,2011-09-01,1,12346.0,2011-01-18 10:01:00,2011-01-01
43731,2011-10-01,1,12346.0,2011-01-18 10:01:00,2011-01-01


In [0]:
report[report['CustomerID'] == 12448.0]

,month,key,CustomerID,reg_date,reg_month
43814,2011-10-01,1,12448.0,2011-10-26 13:46:00,2011-10-01
48187,2011-11-01,1,12448.0,2011-10-26 13:46:00,2011-10-01
52560,2011-12-01,1,12448.0,2011-10-26 13:46:00,2011-10-01


Now we can join sales information. But before doing it

# 3. Pre-aggregate transactional data before joining with report

I will calculate total sales for each month for each customer

In [0]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [0]:
df['month'] = df['InvoiceDate'].values.astype('datetime64[M]')

In [0]:
df['revenue'] = df['UnitPrice'] * df['Quantity']

In [0]:
sales_month = df.groupby(['CustomerID','month'])[['revenue']].agg(['sum']).reset_index()

In [0]:
sales_month.head()

,CustomerID,month,revenue
,,,sum
0,-999.0,2010-12-01,194353.00
1,-999.0,2011-01-01,84925.88
2,-999.0,2011-02-01,61516.50
3,-999.0,2011-03-01,103302.47
4,-999.0,2011-04-01,67159.27


In [0]:
sales_month.columns = ['CustomerID','month','revenue']

In [0]:
sales_month.head()

,CustomerID,month,revenue
0,-999.0,2010-12-01,194353.00
1,-999.0,2011-01-01,84925.88
2,-999.0,2011-02-01,61516.50
3,-999.0,2011-03-01,103302.47
4,-999.0,2011-04-01,67159.27


# 4. Join to report

In [0]:
report = report.merge(sales_month,how='left',on=['CustomerID','month'])

It is very important to use the left join! Using the left join we are keeping all rows from the report table -- that's how we understand gaps in customer activity.

In [0]:
report.head()

,month,key,CustomerID,reg_date,reg_month,revenue
0,2010-12-01,1,-999.0,2010-12-01 11:52:00,2010-12-01,194353.00
1,2010-12-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,711.79
2,2010-12-01,1,12348.0,2010-12-16 19:09:00,2010-12-01,892.80
3,2010-12-01,1,12370.0,2010-12-14 12:58:00,2010-12-01,1868.02
4,2010-12-01,1,12377.0,2010-12-20 09:37:00,2010-12-01,1001.52


Let's look at a particular customer

In [0]:
report[report['CustomerID'] == 12347.0]

,month,key,CustomerID,reg_date,reg_month,revenue
1,2010-12-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,711.79
951,2011-01-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,475.39
2321,2011-02-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN
4071,2011-03-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN
6261,2011-04-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,636.25
8750,2011-05-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN
11518,2011-06-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,382.52
14521,2011-07-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN
17715,2011-08-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,584.91
21076,2011-09-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN


We can see that his activity is not regular! For example there is pause in purchases after 2011-01-01 for two months. And then purchases start again.

# 5. Basic metrics

Add user, active, and new user counts

In [0]:
report['user'] = 1
report['new'] = (report['reg_month'] == report['month']) * 1
report['active'] = (report['revenue'] > 0) * 1

In [0]:
report.head()

,month,key,CustomerID,reg_date,reg_month,revenue,user,new,active
0,2010-12-01,1,-999.0,2010-12-01 11:52:00,2010-12-01,194353.00,1,1,1
1,2010-12-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,711.79,1,1,1
2,2010-12-01,1,12348.0,2010-12-16 19:09:00,2010-12-01,892.80,1,1,1
3,2010-12-01,1,12370.0,2010-12-14 12:58:00,2010-12-01,1868.02,1,1,1
4,2010-12-01,1,12377.0,2010-12-20 09:37:00,2010-12-01,1001.52,1,1,1


In [0]:
report.groupby('month')[['revenue','user','new','active']].agg('sum')

,revenue,user,new,active
month,,,,
2010-12-01,748957.020,949,949,882
2011-01-01,560000.260,1370,421,737
2011-02-01,498062.650,1750,380,754
2011-03-01,683267.080,2190,440,969
2011-04-01,493207.121,2489,299,851
2011-05-01,723333.510,2768,279,1053
2011-06-01,691123.120,3003,235,987
2011-07-01,681300.111,3194,191,943
2011-08-01,682680.510,3361,167,931


Add average revenue per customer

In [0]:
t = report.groupby('month')[['revenue','user','new','active']].agg('sum')

In [0]:
t['avg_revenue'] = t['revenue'] / t['active']

In [0]:
t

,revenue,user,new,active,avg_revenue
month,,,,,
2010-12-01,748957.020,949,949,882,849.157619
2011-01-01,560000.260,1370,421,737,759.837531
2011-02-01,498062.650,1750,380,754,660.560544
2011-03-01,683267.080,2190,440,969,705.125986
2011-04-01,493207.121,2489,299,851,579.561834
2011-05-01,723333.510,2768,279,1053,686.926410
2011-06-01,691123.120,3003,235,987,700.226059
2011-07-01,681300.111,3194,191,943,722.481560
2011-08-01,682680.510,3361,167,931,733.276595


# 6. Churn

Same way you can calculate churn metrics, like number of customers who are inactive for 1,2,3,..,n months. Let's check all logic on this one customer (it's a good habit by the way to check your logic on one object and then scale it to the whole base)

In [0]:
report[report['CustomerID'] == 12347.0]

,month,key,CustomerID,reg_date,reg_month,revenue,user,new,active
1,2010-12-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,711.79,1,1,1
951,2011-01-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,475.39,1,0,1
2321,2011-02-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0
4071,2011-03-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0
6261,2011-04-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,636.25,1,0,1
8750,2011-05-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0
11518,2011-06-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,382.52,1,0,1
14521,2011-07-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0
17715,2011-08-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,584.91,1,0,1
21076,2011-09-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0


For each customer I want to get a marker (or a flag) whether or not he was active in the previous month


In [0]:
report['active_prev'] = (report.sort_values(by=['month'], ascending=True)
                       .groupby(['CustomerID'])['active'].shift(1))

This hard stuff is something like a window function from sql. Because I have many customers I have to somehow say to pandas that they should be treated separately, so that's what groupby for. And also I want months to be in order, that's what sort_values for. Finally shift gives the value from the previous row (month in this case). Check it

In [0]:
report[report['CustomerID'] == 12347.0]

,month,key,CustomerID,reg_date,reg_month,revenue,user,new,active,active_prev
1,2010-12-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,711.79,1,1,1,NaN
951,2011-01-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,475.39,1,0,1,1.0
2321,2011-02-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0
4071,2011-03-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,0.0
6261,2011-04-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,636.25,1,0,1,0.0
8750,2011-05-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0
11518,2011-06-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,382.52,1,0,1,0.0
14521,2011-07-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0
17715,2011-08-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,584.91,1,0,1,0.0
21076,2011-09-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0


Correct. Compare the active column with active_prev. Now I want to understand when customer go from active to inactive and vice versa

In [0]:
report['change_status'] = (report['active'] != report['active_prev']) * 1

In [0]:
report[report['CustomerID'] == 12347.0]

,month,key,CustomerID,reg_date,reg_month,revenue,user,new,active,active_prev,change_status
1,2010-12-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,711.79,1,1,1,NaN,1
951,2011-01-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,475.39,1,0,1,1.0,0
2321,2011-02-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0,1
4071,2011-03-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,0.0,0
6261,2011-04-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,636.25,1,0,1,0.0,1
8750,2011-05-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0,1
11518,2011-06-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,382.52,1,0,1,0.0,1
14521,2011-07-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0,1
17715,2011-08-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,584.91,1,0,1,0.0,1
21076,2011-09-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0,1


Looks good. Next I want to assign some kind of id to each status change. And here is a very neat trick with cumulative sum. Look at the change_status column. If I perform a cumulative sum on this column I will get an increment each time the user changes status. That's what I want. And again I use this scary construction because I want pandas to treat each user separately.

In [0]:
report['session_id'] = (report.sort_values(by=['month'], ascending=True)
                       .groupby(['CustomerID'])['change_status'].cumsum())

In [0]:
report[report['CustomerID'] == 12347.0]

,month,key,CustomerID,reg_date,reg_month,revenue,user,new,active,active_prev,change_status,session_id
1,2010-12-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,711.79,1,1,1,NaN,1,1
951,2011-01-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,475.39,1,0,1,1.0,0,1
2321,2011-02-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0,1,2
4071,2011-03-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,0.0,0,2
6261,2011-04-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,636.25,1,0,1,0.0,1,3
8750,2011-05-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0,1,4
11518,2011-06-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,382.52,1,0,1,0.0,1,5
14521,2011-07-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0,1,6
17715,2011-08-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,584.91,1,0,1,0.0,1,7
21076,2011-09-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0,1,8


It's perfect. Find a session with id 2. Can you see that it is the session where the user was inactive for two months? Now I can sum up the number of inactive months per user and per each session.

In [0]:
report['inactive'] = (report['active'] == 0) * 1

In [0]:
report['month_inactive'] = (report.sort_values(by=['month'], ascending=True)
                       .groupby(['CustomerID','session_id'])['inactive'].cumsum())

Look at row number 4 (with index of 4067). In the month_inactive column we get the correct number of months the customer was inactive. Check it with your eyes. And it's correct for the whole table).

In [0]:
report[report['CustomerID'] == 12347.0]

,month,key,CustomerID,reg_date,reg_month,revenue,user,new,active,active_prev,change_status,session_id,inactive,month_inactive
1,2010-12-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,711.79,1,1,1,NaN,1,1,0,0
951,2011-01-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,475.39,1,0,1,1.0,0,1,0,0
2321,2011-02-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0,1,2,1,1
4071,2011-03-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,0.0,0,2,1,2
6261,2011-04-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,636.25,1,0,1,0.0,1,3,0,0
8750,2011-05-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0,1,4,1,1
11518,2011-06-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,382.52,1,0,1,0.0,1,5,0,0
14521,2011-07-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0,1,6,1,1
17715,2011-08-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,584.91,1,0,1,0.0,1,7,0,0
21076,2011-09-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0,1,8,1,1


Now let's calculate the metric. Say's number of users who has been in churn for two or more months

In [0]:
report['churn_2m'] = (report['month_inactive'] >= 2) * 1

In [0]:
report[report['CustomerID'] == 12347.0]

,month,key,CustomerID,reg_date,reg_month,revenue,user,new,active,active_prev,change_status,session_id,inactive,month_inactive,churn_2m
1,2010-12-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,711.79,1,1,1,NaN,1,1,0,0,0
951,2011-01-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,475.39,1,0,1,1.0,0,1,0,0,0
2321,2011-02-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0,1,2,1,1,0
4071,2011-03-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,0.0,0,2,1,2,1
6261,2011-04-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,636.25,1,0,1,0.0,1,3,0,0,0
8750,2011-05-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0,1,4,1,1,0
11518,2011-06-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,382.52,1,0,1,0.0,1,5,0,0,0
14521,2011-07-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0,1,6,1,1,0
17715,2011-08-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,584.91,1,0,1,0.0,1,7,0,0,0
21076,2011-09-01,1,12347.0,2010-12-07 14:57:00,2010-12-01,NaN,1,0,0,1.0,1,8,1,1,0


Looks good. Now groupby to get info for all customers per month

In [0]:
t = report.groupby('month')[['revenue','user','new','active','churn_2m']].agg('sum')

In [0]:
t

,revenue,user,new,active,churn_2m
month,,,,,
2010-12-01,748957.020,949,949,882,0
2011-01-01,560000.260,1370,421,737,55
2011-02-01,498062.650,1750,380,754,509
2011-03-01,683267.080,2190,440,969,749
2011-04-01,493207.121,2489,299,851,966
2011-05-01,723333.510,2768,279,1053,1225
2011-06-01,691123.120,3003,235,987,1367
2011-07-01,681300.111,3194,191,943,1619
2011-08-01,682680.510,3361,167,931,1871


You also may want to get it in %. Choice of denominator depends on your methodology of churn. Here I will just use current user count for each month. But it might be user count for the previous month or even for month-2, because we are measuring two month churn.

In [0]:
t['churn_2m%'] = t['churn_2m'] / t['user']

In [0]:
t

,revenue,user,new,active,churn_2m,churn_2m%
month,,,,,,
2010-12-01,748957.020,949,949,882,0,0.000000
2011-01-01,560000.260,1370,421,737,55,0.040146
2011-02-01,498062.650,1750,380,754,509,0.290857
2011-03-01,683267.080,2190,440,969,749,0.342009
2011-04-01,493207.121,2489,299,851,966,0.388108
2011-05-01,723333.510,2768,279,1053,1225,0.442558
2011-06-01,691123.120,3003,235,987,1367,0.455211
2011-07-01,681300.111,3194,191,943,1619,0.506888
2011-08-01,682680.510,3361,167,931,1871,0.556680


In the project you can use this table to calculate revenue from tariffs information. Just add tariff column for each user and then join tariffs table and calculate revenue in new column

This is a very robust way to create all kinds of reports: daily, weekly, monthly. If you generate all possible dates you will miss nothing. But keep in mind that the operation of cross join is computationally expensive and better be done in some sql database. I have a video about it on my channel https://youtu.be/s2uA0CTAlH0